In [ ]:
# imports and setups
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Define the train and test data paths
base_dir = '../../data/raw/fer2013/'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Emotion class configuration
original_classes = ['angry', 'disgust', 'fear',
                    'happy', 'sad', 'surprise', 'neutral']
target_classes = ['happy', 'sad', 'angry', 'stressed', 'neutral']

class_mapping = {
    'angry': 'angry',
    'disgust': 'angry',      # Merge disgust into angry
    'fear': 'stressed',      # Map fear to stressed
    'happy': 'happy',
    'sad': 'sad',
    'surprise': 'neutral',   # Map surprise to neutral
    'neutral': 'neutral'
}

In [ ]:
# Custom data generating
class MergedDataGenerator(ImageDataGenerator):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def flow_from_directory(self, directory, **kwargs):
        generator = super().flow_from_directory(
            directory,
            target_size=(48, 48),
            color_mode='grayscale',
            class_mode='categorical',
            batch_size=64,
            shuffle=True,
            **kwargs
        )

        while True:
            x_batch, y_batch = next(generator)
            y_merged = np.zeros((y_batch.shape[0], 5))

            for i in range(y_batch.shape[0]):
                original_class = np.argmax(y_batch[i])
                original_name = original_classes[original_class]
                target_name = class_mapping[original_name]
                target_index = target_classes.index(target_name)
                y_merged[i, target_index] = 1

            yield x_batch, y_merged


# Initialize generators
train_datagen = MergedDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = MergedDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir)
test_generator = test_datagen.flow_from_directory(test_dir)

In [ ]:
# Model Architecture
def build_model():
    model = Sequential([
        # Block 1
        Conv2D(64, (3, 3), activation='relu',
               padding='same', input_shape=(48, 48, 1)),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Block 2
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Block 3
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        # Classifier
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(5, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


model = build_model()
model.summary()

In [ ]:
# Training process
def train_model(model, train_gen, test_gen):
    callbacks = [
        EarlyStopping(patience=10, restore_best_weights=True),
        ModelCheckpoint('best_model.h5', save_best_only=True)
    ]

    history = model.fit(
        train_gen,
        steps_per_epoch=train_gen.samples // train_gen.batch_size,
        epochs=50,
        validation_data=test_gen,
        validation_steps=test_gen.samples // test_gen.batch_size,
        callbacks=callbacks
    )
    return history


history = train_model(model, train_generator, test_generator)

In [ ]:
# Evaluation Metrics
def evaluate_model(model, test_gen):
    test_gen.reset()
    y_pred = model.predict(test_gen)
    y_pred_classes = np.argmax(y_pred, axis=1)

    y_true = []
    for i in range(len(test_gen)):
        _, labels = test_gen[i]
        y_true.extend(np.argmax(labels, axis=1))
    y_true = np.array(y_true)[:len(y_pred_classes)]

    # Classification report
    print("Classification Report:")
    print(classification_report(y_true, y_pred_classes, target_names=target_classes))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=target_classes,
                yticklabels=target_classes)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()


evaluate_model(model, test_generator)

In [ ]:
# Real Time emotion detection
def real_time_emotion_detection(model):
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            try:
                face_roi = gray[y:y+h, x:x+w]
                resized = cv2.resize(face_roi, (48, 48))
                normalized = resized / 255.0
                reshaped = normalized.reshape(1, 48, 48, 1)

                pred = model.predict(reshaped, verbose=0)
                emotion_idx = np.argmax(pred)
                confidence = np.max(pred)

                if confidence > 0.7:
                    emotion = target_classes[emotion_idx]
                    label = f"{emotion} ({confidence:.2f})"
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x, y-10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            except Exception as e:
                print(f"Processing error: {e}")
                continue

        cv2.imshow('Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Save the model
def save_model(model, filename='emotion_detection_5class.h5'):
    model.save(filename)
    print(f"Model saved as {filename}")


save_model(model)